In [1]:
#local imports
from func.DataLoader import gdfLoader,ReadMarkerMap,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from func.FoliumFeatureCreation.RawDataFeatures import RawDataFeatures
from func.MapCreation import MapCreation
from API import API
from secret import API_KEY,MAC_WEAR
from datetime import datetime
import json
import pandas as pd
from shapely.geometry import Point,Polygon,mapping
import folium
from folium.plugins import TimestampedGeoJson
import numpy as np




In [2]:
MapDataPath="data/cartePalaiseau.csv"
ModuleMapPath="data/carteModulePalaiseau.csv"
Data_Path="data/manipstatic.csv"

saving_output=False

sampling_time=2000#ms #si supérieur à 2000ms il faut implémenter la moyenne dans la liste des rssi dans marker dict list

In [3]:
#Map Data
map_gdf=gdfLoader(MapDataPath)
map_gdf.to_crs(epsg=4326, inplace=True)
#marker data
marker_gdf=ReadMarkerMap(ModuleMapPath)
#rssi data
rssi_df=RSSIDataLoader(Data_Path)

In [4]:
m=MapCreation(map_gdf,marker_gdf=marker_gdf)
m.save('output/map.html')
m

In [5]:
map_gdf.head()

,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ..."


In [6]:
expectation=json.loads(open('data/spf_calibration/calib1.json').read())

In [7]:
# here we get the actual data we want to predict position in zero shot
start,end="2023-06-23 10:42:00.000000","2023-06-23 10:50:10.000000"# should be 4A439

Gatien_API=API(API_KEY)
hour_correction=2
start=pd.to_datetime(start)-pd.Timedelta(hour_correction, unit="h")
end=pd.to_datetime(end)-pd.Timedelta(hour_correction, unit="h")
data=Gatien_API.getRawDataForCartoWear(MAC_WEAR,start,end)
data=pd.DataFrame(data)
data["timestamp"]=pd.to_datetime(data['timestamp'])
data


,timestamp,macModule,rssi
0,2023-06-23 08:50:09.539,A8032A311F96,-61
1,2023-06-23 08:50:09.303,A8032A311DAA,-72
2,2023-06-23 08:50:06.992,A8032A311F96,-65
3,2023-06-23 08:50:06.755,A8032A311DAA,-72
4,2023-06-23 08:50:06.167,A8032A311F6A,-55
...,...,...,...
740,2023-06-23 08:42:03.619,A8032A311FAA,-66
741,2023-06-23 08:42:02.726,A8032A311F96,-65
742,2023-06-23 08:42:02.400,A8032A31204A,-73
743,2023-06-23 08:42:01.377,A8032A311F6A,-55


In [8]:
#parse data to create a list of data for sampled every sampling time ms
timestamp_list= pd.date_range(start=start, end=end, freq=str(sampling_time)+"ms").tolist()
data_list=[]
for i in range(len(timestamp_list)):
    data_treated = data[(data["timestamp"] > timestamp_list[i-1]) & (data["timestamp"] <= timestamp_list[i])]
    data_treated=data.groupby("macModule").mean().reset_index()
    data_treated=data_treated[["macModule","rssi"]]
    data_list.append(data_treated)
data_list    

[      macModule       rssi
 0  A8032A311DAA -71.717105
 1  A8032A311F6A -54.572414
 2  A8032A311F96 -63.557143
 3  A8032A311FAA -66.050360
 4  A8032A31204A -72.475524
 5  A8032A31204E -79.538462,
       macModule       rssi
 0  A8032A311DAA -71.717105
 1  A8032A311F6A -54.572414
 2  A8032A311F96 -63.557143
 3  A8032A311FAA -66.050360
 4  A8032A31204A -72.475524
 5  A8032A31204E -79.538462,
       macModule       rssi
 0  A8032A311DAA -71.717105
 1  A8032A311F6A -54.572414
 2  A8032A311F96 -63.557143
 3  A8032A311FAA -66.050360
 4  A8032A31204A -72.475524
 5  A8032A31204E -79.538462,
       macModule       rssi
 0  A8032A311DAA -71.717105
 1  A8032A311F6A -54.572414
 2  A8032A311F96 -63.557143
 3  A8032A311FAA -66.050360
 4  A8032A31204A -72.475524
 5  A8032A31204E -79.538462,
       macModule       rssi
 0  A8032A311DAA -71.717105
 1  A8032A311F6A -54.572414
 2  A8032A311F96 -63.557143
 3  A8032A311FAA -66.050360
 4  A8032A31204A -72.475524
 5  A8032A31204E -79.538462,
       macModul

In [9]:
def PossibleRoom(data_treated,returnMacModule=False):
    room_dict={}
    for index,row in data_treated.iterrows():
        room_value=expectation[row["macModule"]]
        for room in room_value:
            if room_value[room]:
                min,max=room_value[room]
                if min<=row["rssi"]<=max:
                    if returnMacModule:room_dict[room]=room_dict.get(room,[])+[row["macModule"]]
                    else: room_dict[room]=room_dict.get(room,0)+1
    return(room_dict)


In [10]:
possible_room_list=[]
for i in data_list:
    possible_room_list.append((PossibleRoom(i,returnMacModule=True)))

In [11]:
map_gdf.head()

,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ..."


In [12]:
macModule_list=marker_gdf["macModule"].to_list()


In [13]:
from matplotlib import cm
import matplotlib.colors as colors
n_colors = len(macModule_list)
colours = cm.rainbow(np.linspace(0, 1, n_colors))
#transform to hex
colours = [colors.rgb2hex(i) for i in colours]
macModule_color_dict={}
for index,macmodule in enumerate(macModule_list):
    macModule_color_dict[macmodule]=colours[index]
macModule_color_dict

{'A8032A31204E': '#8000ff',
 'C45BBE39F42A': '#7019ff',
 'C45BBE39F9AE': '#5e35fe',
 'A8032A311FAA': '#4e4dfc',
 'C45BBE39F56A': '#3c68f9',
 'C45BBE37B346': '#2c7ef7',
 'C45BBE386952': '#1996f3',
 'C45BBE386362': '#09a9ee',
 'C45BBE384CD2': '#08bee9',
 'C45BBE39870E': '#18cde4',
 'A8032A312052': '#2adddd',
 'A8032A31204A': '#3ae8d6',
 'A8032A311F6A': '#4df3ce',
 'A8032A311F56': '#5df9c7',
 'C45BBE38FEAA': '#6efebe',
 'A8032A30FBFE': '#80ffb4',
 'A8032A311DAA': '#90feab',
 'A8032A30FB9E': '#a2f9a0',
 'C45BBE39F4D6': '#b2f396',
 'A8032A311F96': '#c4e88a',
 'A8032A311F66': '#d4dd80',
 'A8032A311FBA': '#e6cd73',
 'A8032A311F52': '#f6be68',
 'A8032A311DBA': '#ffa95b',
 'A8032A30FBEE': '#ff964f',
 'A8032A311DD2': '#ff7e41',
 'A8032A30FBDE': '#ff6835',
 'A8032A30FBE2': '#ff4d27',
 'A8032A30FBA2': '#ff351b',
 'A8032A311F8A': '#ff190d',
 'C45BBE3820DA': '#ff0000'}

In [14]:
features = []
input_format = '%Y-%m-%d %H:%M:%S'
output_format = '%Y-%m-%dT%H:%M:%S'


for index,room_dict in enumerate(possible_room_list):
    temp_timestamp=timestamp_list[index]
    dt = datetime.strptime(str(temp_timestamp), input_format)
    timestamp = dt.strftime(output_format)
    for key in room_dict:
        for MacModule in room_dict[key]:
            temp=map_gdf[map_gdf["ID_element"]==key]
            coord=temp['geometry']
            coordinates=mapping(coord)['features'][0]['geometry']['coordinates'][0]  
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': [coordinates],
                },
                'properties': {
                    'times': [timestamp],
                    'style': {
                'color': macModule_color_dict[MacModule],
                    } }}
            features.append(feature)



In [15]:
zone_layer=TimestampedGeoJson(
                    {"type": "FeatureCollection","features": features},
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,
                    auto_play=True)

In [18]:
m=MapCreation(map_gdf,location=[48.711,2.208])

zone_layer.add_to(m)
folium.LayerControl().add_to(m)

m.save('output/spfmap.html')
m

UnboundLocalError: local variable 'marker_gdf' referenced before assignment